# Introduction

Certaines choses étaient auparavant impossibles, et ce n’était peut-être pas si mal : une peinture rupestre représentant un dragon, un papyrus égyptien sensibilisant au réchauffement climatique, une stèle maya honorant Jair Bolsonaro, un paysage de Van Gogh représentant un parc éolien ou encore le portrait de Shrek version Andy Warhol. Les réseaux de neurones offrent aujourd’hui de nombreuses possibilités, comme celles de réaliser de tels anachronismes. Cela est possible grâce à la fusion esthétique de divers contenus visuels avec d’autres styles artistiques. 

Cette méthode de traitement d'image, appelée transfert de style, consiste à métamorphoser l'apparence d'une image en préservant son contenu visuel pour adopter le style artistique d'une autre image de référence.  

SCHEMA IMAGE DE CONTENU + STYLE = NOUVELLE CREA 

 

Le transfert de style trouve des utilités dans des domaines divers grâce à sa capacité à créer des œuvres d'art uniques. Ces créations peuvent être largement exploitées dans les domaines de l'édition d'images et de la publicité. De plus, le transfert de style s'étend aujourd’hui au-delà de l'image statique, ce qui ouvre la porte à tous les domaines l’audiovisuel (ex. Cinéma) et même ceux de la réalité virtuelle et de la réalité augmentée. 

Divers procédés ont été mis en place pour réaliser ce transfert de style, chacun offre un apport dans le domaine malgré les différentes contraintes qu’il présente. L’article étudié est intitulé “Arbitrary Style Transfer in Real-time with Adaptive Instance Normalization” et a été écrit par Xun Huang et Serge Belongie. Publié en 2017 par le Département de Computer Science de Cornell University, il propose une méthode de transfert de style arbitraire en temps réel novatrice. Le terme “arbitraire” signifie que le choix du style pris en entrée n’est pas restreint à un ensemble prédéfini de styles spécifiques. Cette caractéristique permet aux utilisateurs de choisir librement le style qu'ils souhaitent appliquer à leur image et élargit ainsi la portée des applications de l'algorithme en offrant une plus grande liberté créative. 

La méthode appliquée dans l’article étudiée est basée sur une couche de normalisation d'instance adaptative (AdaIN) qui aligne la moyenne et la variance des caractéristiques du contenu avec celles du style. Cette approche atteint une vitesse comparable aux méthodes les plus rapides existantes, tout en éliminant la restriction à un ensemble prédéfini de styles. 

Ce Notebook vise à présenter le fonctionnement de l’algorithme de transfert de style arbitraire en temps réel ainsi que ses limitations. Nous avons aussi ajouté la possibilité de fournir en entrée du réseau des images en noir et blanc ce qui nous a permis d’élargir l’étude des limites de l’algorithme. Dans une première section, nous présentons une étude approfondie de la littérature scientifique disponible sur le sujet. Nous analyserons ensuite dans une seconde section le fonctionnement de la couche de normalisation d'instance adaptative (AdaIN) et la comparerons avec les autres méthodes existantes. La troisième section sera dédiée à la description de l’architecture du réseau de neurone utilisé. Enfin, une étude des limites de la méthode est présentée en quatrième section. CECI EST UN EXEMPLE DE PLAN, MODIFIER SI BESOIN 

# Etat de l'art

Les réseaux neuronaux profonds (DNN) encodent à la fois le contenu et le style de l'image. Il est donc possible d’identifier et de séparer les informations relatives au style et au contenu pour créer de nouvelles images.  

(Gatys et al., 2016) introduit un algorithme neuronal utilisant des réseaux de neurones convolutionnels (CNN). L’activation de certaines couches de neurones permet de capturer le contenu de l’image. Le style, quant-à-lui, est déduit des corrélations statistiques des activations des couches. Pour réaliser le transfert de style, (Gatys et al., 2016) minimise la différence de contenu par rapport à l’image originale tout en recréant les statistiques du style souhaité. La méthode parvient à transférer le style souhaité sur le contenu initial, cependant, elle utilise un processus d'optimisation itératif lent et nécessite des images similaires en entrée. 

Plusieurs articles se consacrent à accélérer le processus de transfert. (J.Johnson, 2016) notamment entraîne réseau neuronal convolutionnel feedforward. L’approche adoptée est basée sur le fait que la perception visuelle humaine est davantage axée sur des caractéristiques sémantiques et structurelles que sur des détails pixel à pixel. Ainsi, plutôt que de s’intéresser aux différences pixel à pixel, la méthode utilise les pertes perceptuelles pour mesurer la différence perceptuelle entre les images générées et les images de référence. Ces pertes perceptuelles sont obtenues grâce à des réseaux de neurones pré-entrainés, tels que VGG-19. Le processus d'optimisation permet d’obtenir des résultats en temps réel mais chaque réseau reste limité à un unique style.  

Les méthodes affranchies cette limitation sont soit restreintes à un ensemble fini de styles ou soit beaucoup plus lentes que les méthodes de transfert à style unique. Par exemple, (Y. Li, 2017) se concentre sur la reproduction des textures plutôt que de motifs spécifiques. L’utilisation de réseaux de neurones feedforward produit une image de sortie en temps réel mais la méthode reste restreinte à un ensemble fini de textures. 

# Notions fondamentales

Dans le contexte des réseaux de neurones traitant des données bidimensionnelles (2D), comme les images, les données d'entrée sont habituellement représentées sous la forme d'un tenseur à quatre dimensions. Ce tenseur est noté $(N_{\text{total}}, C, H, W)$, où $N_{\text{total}}$ représente le nombre total d'images, $C$ le nombre de canaux (par exemple, 3 pour une image RGB), et $H$ et $W$ désignent respectivement la hauteur et la largeur de chaque image.

Un échantillon unique dans ce contexte correspond à une donnée 2D, c’est-à-dire une image, qui a pour dimension $(C, H, W)$.

Pour améliorer l'efficacité et la performance des réseaux de neurones, il est courant de regrouper les données en 'batches'. Un batch est un sous-ensemble du jeu de données complet et est de dimension $(N, C, H, W)$, où $N$ indique la taille du batch, c’est-à-dire le nombre d'échantillons traités ensemble lors d'une seule étape de l'apprentissage.

Nous utiliserons ces notations et concepts tout au long de notre explication.

# Définition et calcul

## Normalisation

La normalisation des données joue un rôle crucial dans l'apprentissage des réseaux de neurones profonds. Bien que la normalisation à l'entrée soit essentielle, elle s'avère souvent insuffisante en raison d'un phénomène connu sous le nom de 'gradient evanescent'. Ce dernier se manifeste par une réduction progressive de l'amplitude du gradient lors de la rétropropagation, particulièrement due aux faibles valeurs de sortie des couches d'activation, comme les fonctions sigmoid ou tanh. Dans les réseaux comportant de nombreuses couches, la multiplication répétée de ces petits termes peut entraîner la diminution significative du gradient.

Pour contrer ce problème, une renormalisation des données à travers les différentes couches du réseau s'avère nécessaire. De plus, un autre défi, connu sous le nom de 'décalage covariant interne', survient. Ce phénomène résulte de la modification de la distribution des activations à chaque couche, due aux transformations non linéaires appliquées par le réseau. Autrement dit, même si les données en entrée présentent une moyenne de 0 et une variance de 1, ces caractéristiques se déforment au fur et à mesure qu'elles traversent les différentes couches du réseau. Des techniques telles que Batch Normalization ont été développées pour répondre à ces enjeux.

Bien que les objectifs de Batch Normalization et de Instance Normalization, telle que proposée dans le papier en question, soient différents (la première visant à renormaliser les données pour éviter les problèmes précédemment décrits, tandis que la seconde a pour but de neutraliser le contrast et la variation des couleurs au sein d'une image), une étude de Batch Normalization reste pertinente. En effet, dans les deux cas, il s'agit d'une forme de normalisation des données, bien que leurs applications et finalités soient distinctes.

La formule générale pour la normalisation dans les réseaux de neurones est la suivante :

$$ Normalisation(x)=\gamma \left(\frac{x-\mu(x)}{\sigma(x)}\right)+\beta, $$

où $x$ est une image, $\gamma, \beta$ sont des paramètres affines extraits des données (les paramètres d'échelle et de décalage resp.). 

La différence dans les formules entre Instance Normalization et Batch Normalization vient dans la définition de $\mu(x), \sigma(x)$. 

## Batch normalization (BN)

Batch Normalization consiste à normaliser les activations d'un réseau de neurones pour chaque lot (ou batch) de données traitées. Concrètement, au lieu de normaliser une seule image (ou échantillon), cette technique ajuste les activations de toutes les images dans un batch en fonction de la moyenne et de la variance calculées sur l'ensemble du batch, et ce, pour chaque canal de manière indépendante, comme illustré sur la figure ci-dessous. Ainsi, pour chaque canal, les activations sont d'abord centrées autour de zéro (en soustrayant la moyenne du batch) puis mises à l'échelle (en divisant par l'écart-type du batch).

Les expressios de la moyenne et de l'écart-type de BN sont les suivantes : 

$$\mu_{batch}(x) = \frac{1}{NHW}\sum_{n=1}^{N}\sum_{h=1}^{H}\sum_{w=1}^{W}x_{nchw},$$

$$
\sigma_{batch}(x) = \sqrt{\frac{1}{NHW} \sum_{n=1}^{N} \sum_{h=1}^{H} \sum_{w=1}^{W} \left(x_{nchw} - \mu_{batch}(x)\right)^2+\epsilon}, 
$$

où $x_{nchw}$ est une image de batch $n$ de taille $h$ x $w$ et de canal $c$. 


![BN](/resources/images/BN.png)

## Instance normalization (IN)

L'Instance Normalization, ou normalisation d'instance, consiste à normaliser chaque échantillon individuellement en fonction de sa propre moyenne et variance pour chaque canal. Concrètement, cela signifie que pour chaque image dans un batch, la normalisation est effectuée séparément, canal par canal. Cette approche est particulièrement utile dans les tâches où l'indépendance entre les échantillons est cruciale, comme dans le transfert de style.

Les expressios de la moyenne et de l'écart-type de IN sont les suivantes : 

$$\mu_{image}(x) = \frac{1}{HW}\sum_{h=1}^{H}\sum_{w=1}^{W}x_{chw},$$

$$
\sigma_{image}(x) = \sqrt{\frac{1}{HW} \sum_{h=1}^{H} \sum_{w=1}^{W} \left(x_{chw} - \mu_{image}(x)\right)^2+\epsilon}, 
$$

où $x_{chw}$ est une image de taille $h$ x $w$ et de canal $c$. 


![IN](/resources/images/IN.png)

## Conditional Instance Normalization (CIN)

La Conditional Instance Normalization (CIN) est une extension de la méthode d'Adaptive Instance Normalization (AIN) présentée dans l'article. La CIN introduit une condition supplémentaire qui permet d'ajuster la normalisation en fonction d'une information conditionnelle.

On a donc :

$$
CIN(x;s) = \gamma^s \left(\frac{x-\mu(x)}{\sigma(x)}\right) + \beta^s
$$

Avec :

* $x$ : Représente les activations d’une couche de réseau de neurones
* $CIN(x;s)$ : Représente les activations normalisées
* $\mu$ et $\sigma$ : moyenne et l'écart-type des activations de la couche x. Ces valeurs sont utilisées pour normaliser les activations et les rendre centrées et mises à l'échelle
* $\gamma$ et $\beta$ : Paramètres de normalisation affine. Ils sont sous forme de matrice NxC avec N le nombre de styles et C le nombre de feature maps

En d'autres termes, le processus de normalisation conditionnelle (CIN) consiste à ajuster les activations $x$ en fonction du style $s$ en utilisant des **paramètres spécifiques à ce style**.

La moyenne ($\mu$) et l'écart-type ($\sigma$) sont calculés à partir des activations $x$, et ces valeurs sont utilisées pour **normaliser les activations**. Ensuite, les paramètres $\gamma^s$ et $\beta^s$, spécifiques au **style**, sont appliqués pour ajuster davantage les activations normalisées en fonction du style artistique souhaité.

L'aspect **conditionnel** de la CIN réside dans le fait que les paramètres $\gamma$ et $\beta$ sont **conditionnés par le style de peinture spécifique $s$**. Cela signifie que, pour chaque style $s$, le modèle utilise des **paramètres de normalisation différents**, il peut maintenant non seulement apprendre à adapter le style en fonction de l'image source, mais aussi à ajuster ce style en fonction de conditions spécifiques en ajoutant cette condition et devient capable de faire du transfert de style tout en tenant compte d'une caractéristique spécifique définie.

Pendant l'entraînement, une image de style est choisie aléatoirement parmi un ensemble fixe de styles ($s \in \{ 1,2,…,S \}$). Le réseau d'apprentissage de style utilise ensuite les paramètres $\gamma^s$ et $\beta^s$ correspondants dans les couches CIN.
Il est souligné que le réseau peut générer des images dans des styles complètement différents en utilisant les mêmes paramètres de convolution mais des paramètres affines ($\gamma$ et $\beta$) différents dans les couches IN.

Une des limitations de cette méthode est qu'elle ajoute des paramètres proportionnels au nombre de styles, rendant difficile l'extension à un grand nombre de styles sans une augmentation linéaire des paramètres. De plus, cette approche ne peut pas s'adapter à de nouveaux styles sans ré-entraînement, limitant sa flexibilité.

En résumé, bien que la CIN permette d'ajuster le style en fonction de différents ensembles de paramètres, elle présente des limitations en termes de complexité et de flexibilité pour gérer un grand nombre de styles.

## Interpreting Instance Normalisation

Malgré le succès de l'IN, les raisons précises pour lesquelles elle fonctionne particulièrement bien pour le transfert de style ne sont pas complètement comprises. Une hypothèse suggère que l'IN est invariant au contraste de l'image source, mais cette explication semble limitée. L'IN agit dans l'espace des caractéristiques plutôt que dans l'espace des pixels, ce qui suggère que son impact va au-delà d'une simple normalisation du contraste.

Les auteurs soutiennent que la normalisation d'instance effectue une forme de normalisation de style en normalisant les statistiques des caractéristiques, notamment la moyenne et la variance. En effet, les statistiques des caractéristiques d’un DNN sont capables de capturer le style d’une image. Les auteurs pensent que les statistiques des caractéristiques d'un réseau générateur peuvent également contrôler le style de l'image générée.

Les chercheurs ont observé que les paramètres affines (les paramètres qui agissent sur l'étirement et la translation) dans l'IN peuvent altérer complètement le style de l'image de sortie, ce qui est surprenant. Ils soulignent que les statistiques des caractéristiques convolutives d'un réseau de neurones peuvent capturer le style d'une image. L'IN semble effectuer une sorte de normalisation de style en agissant sur ces statistiques, en particulier la moyenne et la variance des caractéristiques.

Les auteurs ont ensuite décidé de comparer cette méthode avec la **Batch Normalization** en exécutant le code Improved Texture Network puis en normalisant le contraste et le style des images d'entraînement :

* Lors du transfert de style, le modèle avec IN converge plus rapidement que celui avec BN

* Même après la normalisation du contraste des images d'entraînement, IN reste efficace

* Lorsque les images d'entraînement sont normalisées vers un style spécifique, l'amélioration apportée par IN diminue, indiquant que la normalisation du style n'est pas parfaite

* Les modèles avec BN, entraînés sur des images avec normalisation de style, peuvent converger aussi rapidement que les modèles avec IN entraînés sur des images originales. Cela suggère que l'IN effectue une forme de normalisation de style, tandis que la BN, bien qu'elle normalise les statistiques des caractéristiques d'un lot, ne garantit pas un transfert global vers un style unique, ce qui est essentiel dans le transfert de style à propagation avant

L'IN peut normaliser le style de chaque échantillon individuel vers le style cible, ce qui facilite l'entraînement. Les autres parties du réseau peuvent alors se concentrer le contenu tout en éliminant l'information de style originale. Cela explique le succès de l'IN dans le transfert de style. De plus, l'explication devient claire pour la Conditional Instance Normalization (CIN) : des paramètres affines différents peuvent normaliser les statistiques des caractéristiques vers des valeurs différentes, entraînant ainsi une normalisation de l'image de sortie vers différents styles.

En résumé, l'IN semble jouer un rôle important dans la normalisation du style pendant le transfert de style, en agissant sur les caractéristiques plutôt que sur les pixels, et en permettant une normalisation fine et spécifique au style pour chaque échantillon individuel.

# Différences dans l'application

La normalisation par batchs suppose que les images dans un batch donné sont représentatives de l'ensemble du jeu de données. Cette hypothèse peut ne pas toujours être vraie, surtout pour les petits batchs.

D'autre part, la normalisation d'instance (Instance Normalization) opère sous l'hypothèse que chaque image possède son propre style. En conséquence, elle normalise les données en fonction de la moyenne et de la variance de chaque échantillon individuel, et ce, pour chaque canal. Cette méthode est particulièrement adaptée à des applications telles que le transfert de style, où le style unique de chaque image est un facteur clé.

Pour la Batch Normalization (BN), les couches utilisent des statistiques de batch (moyenne et variance) durant l'entraînement, et des statistiques de population (moyenne et variance de l'ensemble de données d'apprentissage) durant le test. Ceci est dû à l'hypothèse de BN selon laquelle un batch est représentatif de l'ensemble du jeu de données d'apprentissage, ce qui n'est pas nécessairement le cas dans un contexte de test. Pour améliorer la robustesse, on remplace donc les statistiques du batch par celles de la population durant l'inférence. En revanche, la normalisation d'instance (IN) ne repose pas sur une telle hypothèse. Ainsi, les statistiques restent inchangées entre l'apprentissage et le test. En résumé, pour la BN, les statistiques utilisées durant l'inférence sont fixes (celles tirées de l'ensemble d'entraînement), tandis que pour l'IN, les statistiques sont calculées aussi bien durant l'apprentissage que lors du test.

![BN vs IN](/resources/images/BN_vs_IN.png)

D'après les graphiques présentés ci-dessus, on observe que l'Instance Normalization (IN) contribue significativement à la réduction de la perte de style lorsqu'on entraîne le réseau avec des images originales et celles normalisées en termes de contraste. Toutefois, l'amélioration apportée par l'IN semble moins marquée pour les images préalablement normalisées en fonction de leur style. Cette observation s'aligne avec les explications précédentes : l'un des principaux atouts de l'IN réside dans son adaptation aux cas où chaque image présente un style individuel distinct. Or, dans cette situation, le style intrinsèque des images d'entrée a été normalisé, ce qui réduit l'effet et l'utilité de l'IN. Par conséquent, les améliorations apportées par l'IN dans ce contexte spécifique apparaissent relativement limitées.

## Principe du transfert de style :

 Le transfert de style est un processus permettant de transferer le style d'une image à une image dont l'on souhaite garder le contenue. 

# Architecture

On considére deux images de même taille : une correspondant à l'image contenue notée $\textbf{ImgC}$ et une correspondant à l'image style notée $\textbf{ImgS}$.  Le transfert de style repose sur un mécanisme qui comprend un encodeur, ADAIN et un décodeur. 

## 1. Encodeur 

L'encodeur va permettre d'extraire les $\textbf{features}$ de l'image de contenue $ImgC$  d'une part et les $\textbf{features}$ de l'image de style $ImgS$ d'autre part, que l'on note respectivement $\text{f(ImgC)}$ et $\text{f(ImgS)}$.  Cette encodeur est composé de 13 couches de réseaux de neuronnes dont 10 couches de convolutions et 3 couche de Maxpoolings. Considérons une image d'entrée de dimension (h,w,c), alors on décrit l'encodeur par le tableau suivant : 

![Texte alternatif de l'image](Encodeur.png)



où h, w et c sont respectivement la hauteur, la largeur et le nombre de canaux de l'image en entrée. De plus, les dimensions des cartes fonctionnelles intermédiaires sont données par les formules suivantes:   $w_1$=$[\frac{w}{2}-1]+1$ , $h_1$= $[\frac{h}{2}-1]+1$ , $w_2$=$[\frac{w_1}{2}-1]+1$, $h_2$=$[\frac{h_1}{2}-1]+1$, $w_3$=$[\frac{w_2}{2}-1]+1$ et $h_3$=$[\frac{h_2}{2}-1]+1$

$\textbf{Remarque} :$

L’encodeur transforme l’entrée en une représentation dans un espace de dimension plus faible appelé espace latent. L’encodeur compresse donc l’entrée dans une représentation moins coûteuse. 

## 2. ADAIN 

Après l’encodeur, nous connaissons la taille de la carte de fonctionnalités : pour une image d’entrée (w,h,C), nous obtenons une carte de fonctionnalités de dimension ($w_3$, $h_3$, 512) pour l'image de style et de contenue.  L'instance de normalisation adaptative opère en normalisant la carte de fonctionnalités de l'image contenue, représentée par $x = \text{f(ImgC)}$, en se basant sur les statistiques obtenues à partir de la carte de fonctionnalités de l'image style, notée $y = \text{f(ImgS)}$. Nous avons illustrer le mécanisme par le schéma suivant :

![Texte alternatif de l'image](ADAIN.png)


La première étape de ce processus implique le calcul de la moyenne des valeurs des pixels de chaque canal sur toute la largeur et la hauteur du tenseur d'entrée, ainsi que l'écart-type des valeurs des pixels de chaque canal sur l'ensemble de ces dimensions. En résultat, nous obtenons des vecteurs moyennes $\mu_c$ et $\mu_s$ pour chaque canal de la carte de fonctionnalités de contenu et de style, accompagnés de leurs écart-types respectifs notés  $\sigma_c$ et $\sigma_s$, tous ces vecteurs étant de dimension 512.

La seconde étape consiste à normalisé chacun des pixels de la carte de fonctionnalité de contenues par la moyenne $\mu_c^c$ et l'écart-type $\sigma_c^c$ de tout valeurs des pixels qui appartiennent au même canal. Cette valeur va être alignée au valeur de la carte fonctionnalité de style en y multipliant l'écart-type $\sigma_s^c$ de tout valeurs des pixels qui appartiennent au même canal dans la carte de fonctionnalité style et en y ajoutant la moyenne $\mu_s^c$.

A la fin de ces étapes, nous obtenons donc la carte de fonctionalité cible que l'on note t= $ADAIN(x,y)$.

## Extension de Instance normalisation, 

## 2. Décodeur 

A partir du $\textbf{feature de cible}$, c'est-à-dire les features obtenues par la méthode ADAIN, le décodeur va essayer de reconstruire l'image contenue stylisé. Le décodeur est composé de 12 couches de réseaux de neuronnes dont 10 couches de convolutions et  de Upsampling.  

![Texte alternatif de l'image](decodeur.png)

$\textbf{Remarque} $ : 
    On remarque la présence de 3 couche UPSAMPLING permettant de redimensionner ou agrandir le tenseur d'entrée d’un facteur 2, ‘nearest’ indique une interpolation par le plus proche voisin. Cette méthode attribue la valeur du pixel d'entrée le plus proche au pixel correspondant en sortie, en faisant une méthode d'interpolation simple et rapide.

#### Nombre total de paramétres à estimer :

5 240 704 + 3 * c paramétres à estimer 


## Entrainement 

Le modéle est entrainé sur 80 000 images, on utilise l'optimiseur ADAM et le taille de batch est égale à 8.

## Perte liée au contenue

   Cette perte permet de quantifier la perte liées au contenue en effet l’encodeur va permettre d’extraire les features des images de style $f(ImgS)$ et de contenue $f(ImgC)$. Ces features vont être transmise à la couche $ADAIN()$ qui aligne la moyenne et la variance des cartes de fonctionnalités de contenu sur celles de les cartes de caractéristiques de style, produisant les caractéristiques cibles que l'on note t= $ADAIN(f(ImgS), f(ImgC))$. 

![Texte alternatif de l'image](architecture.jpg)

 Ensuite cette carte de fonctionnalités va être reconstruite par un décodeur afin d’avoir une image ayant le contenue et style souhaité g(t) . Ensuite, on va extraire la carte fonctionnelle de cette image dont le contenue à été stylisé, on passe par un encodeur : on obtient les caractéristique de l’image reconstruite ayant le contenue et style souhaité f(g(t)). 


$$ L_c = || f(g(t)) -t||_2$$

La perte du contenue correspond à la différence euclidienne entre les caractéristiques de fonctionnalité à la sortie de l’ADAIN() et celle extraite de l’image reconstruite ayant le contenue et style souhaité, dont on vérifie bien la compatibilité au niveau des dimensions. 

### PRÉ-entrainement ?

## Perte liée au style

Le transfert de style s'effectue en adaptant les statistiques de l'image de contenu pour correspondre à celles de l'image de style, en se concentrant particulièrement sur la moyenne et l'écart-type. Ainsi, il est logique d'utiliser une perte de style qui vise à minimiser la différence entre ces statistiques pour la sortie du réseau et l'image de style de référence. La formule de perte de style présentée dans le papier est la suivante :

$$
L_s = \sum_{i=1}^{L} \left\| \mu(\phi_i(g(t))) - \mu(\phi_i(s)) \right\|^2 + \sum_{i=1}^{L} \left\| \sigma(\phi_i(g(t))) - \sigma(\phi_i(s)) \right\|^2,
$$
où $L$ est le nombre total des couches de VGG-19, $\mu$ et $\sigma$ sont respectivement la moyenne et l'écart-type des activations d'une couche de VGG-19, $s$ est l'image de style de référence et $g(t)$ est l'image stylisée, i.e. la sortie finale du réseau. 

Remarque : Dans leurs expériences, les auteurs ont choisi d'utiliser quatre couches spécifiques du réseau VGG-19.

Le papier mentionne également l'usage courant de la perte de Gram, une autre méthode fréquemment employée dans le transfert de style. Cependant, bien que les résultats obtenus avec cette méthode soient similaires, les auteurs soulignent qu'il est conceptuellement plus approprié d'adopter la perte de style décrite précédemment. Cette approche est en effet plus cohérente avec l'objectif de minimiser les différences entre les statistiques clés (moyenne et écart-type) des images de style et de contenu.

## Limitation